In [ ]:
import math
import os
import numpy as np
import pandas as pd
import torch
import gpytorch
import matplotlib.pyplot as plt
import datetime 
from sklearn.model_selection import train_test_split

# import OrderedDict

In [ ]:
call_exp = {
    "A":"January",
    "B":"February",
    "C":"March",
    "D":"April",
    "E":"May",
    "F":"June",
    "G":"July",
    "H":"August",
    "I":"September",
    "J":"October",
    "K":"November",
    "L":"December"
}
put_exp = {
    "M":"January",
    "N":"February",
    "O":"March",
    "P":"April",
    "Q":"May",
    "R":"June",
    "S":"July",
    "T":"August",
    "U":"September",
    "V":"October",
    "W":"November",
    "X":"December"
}

exp_month = {
    "A":"January",
    "B":"February",
    "C":"March",
    "D":"April",
    "E":"May",
    "F":"June",
    "G":"July",
    "H":"August",
    "I":"September",
    "J":"October",
    "K":"November",
    "L":"December",
    "M":"January",
    "N":"February",
    "O":"March",
    "P":"April",
    "Q":"May",
    "R":"June",
    "S":"July",
    "T":"August",
    "U":"September",
    "V":"October",
    "W":"November",
    "X":"December"
}

In [ ]:
# loading in futures data
import pickle 
import datetime
from scipy.interpolate import interp1d


# open futures.pickle
with open('futures.pickle', 'rb') as handle:
    futures = pickle.load(handle)

name_map = {'H':3, 'M':6, 'U':9, 'Z':12}
future_chain = {}
for es in futures:
    date = str(name_map[es[2]]) + '-' + es[-2:]
    future_chain[date] = futures[es]
    future_chain[date].index = pd.to_datetime(future_chain[date].index) 

def build_fwd_curve(quote_date, underlying):
    # find the futures contract that is closest to the ttm

    for es in future_chain.keys():
        if pd.to_datetime(es, format="%m-%y") > quote_date:
            break
        
    # find next 4 contracts and get prices on quote_date
    dates = [quote_date]
    for i in range(4):
        dates.append(pd.to_datetime(es, format="%m-%y") + pd.DateOffset(months=3*i))
        
    fwd_curve = [underlying]
    for date in dates[1:]:
        price = future_chain[date.strftime("%-m-%y")].loc[quote_date]
        fwd_curve.append(price.iloc[0])

    
    # interpolate to get the price at ttm
    ttm_curve = [0]
    for i in range(1, len(dates)):
        ttm_curve.append((dates[i] - quote_date).days / 365)
    
    fwd_curve = np.array(fwd_curve)
    ttm_curve = np.array(ttm_curve)

    f = interp1d(ttm_curve, fwd_curve, kind='cubic', fill_value='extrapolate')

    return f



In [ ]:
# dumas vol surface
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

def dumas_surface(df):
    X = df['mny']
    T = df['T']
    
    df['X^2'] = X**2
    df['T^2'] = T**2
    df['XT'] = X*T
    
    features = df[['mny', 'X^2', 'T', 'T^2', 'XT']]
    target = df['iv']
    
    model = LinearRegression()
    model.fit(features, target)

    return model

### DATA PARSING

In [ ]:
daily_chains = {}
for year in ['2021', '2022', '2023']:
    
    # load in dataset
    df = pd.read_csv('../euro_option/elektrontimeseries_' + year + '.csv')

    # necessary rows
    df = df[['Ask', 'Bid', 'Last', 'Implied Volatility', 'RIC', 'Trade Date']]

    # get underlying price
    df_spx = df[df['RIC'] == '.SPX']
    spx = df_spx[['Last', 'Trade Date']].copy(deep=True)
    spx['Trade Date'] = pd.to_datetime(spx['Trade Date'])
    spx.index = spx['Trade Date']
    spx.drop(columns=['Trade Date'], inplace=True)

    # get option data
    df_opt = df[(df['RIC'] != '.SPX') & (df['RIC'].str.slice(8,13).str.isnumeric())]
    opts = df_opt.copy(deep=True)
    
    # trade date
    opts['quote'] = pd.to_datetime(opts['Trade Date'])

    # month code
    opts['month_code'] = opts['RIC'].str.slice(3,4)

    # expiration date
    opts['year'] = pd.to_datetime(opts['RIC'].str.slice(6,8), format='%y').dt.year
    opts['month'] = pd.to_datetime(opts['month_code'].str.upper().map(exp_month), format='%B').dt.month
    opts['day'] = pd.to_datetime(opts['RIC'].str.slice(4,6), format='%d').dt.day
    opts['exp'] = pd.to_datetime(opts[['year', 'month', 'day']])
    opts['T'] = opts['exp'] - opts['quote']
    opts['T'] = opts['T'].dt.days

    # option type
    opts['option_type'] = np.where(opts['month_code'].str.upper().isin(call_exp.keys()), 'c', 'p')

    # strike price
    opts['k1000+'] = opts['month_code'].str.islower()
    opts['strike'] = opts['RIC'].str.slice(8,13).astype(float)
    opts['strike'] = np.where(opts['k1000+'], opts['strike']/10, opts['strike']/100)

    # underlying
    opts = opts.merge(spx, left_on='quote', right_on='Trade Date', how='left')
    opts.rename(columns={'Last_x':'last', 'Last_y':'underlying'}, inplace=True)

    # midprice
    opts['midprice'] = (opts['Ask'] + opts['Bid']) / 2

    # build forward curves
    fwd_curves = {}
    for quote_date in opts['quote'].unique():
        fwd_curves[quote_date] = build_fwd_curve(quote_date, opts.loc[opts['quote'] == quote_date, 'underlying'].iloc[0])
    
    opts['fwd_price'] = opts.apply(lambda x: fwd_curves[x['quote']](x['T']/365), axis=1)
    opts['moneyness'] = opts['fwd_price'] / opts['strike']
    
    # converting to ML features
    opts['T'] = opts['T'].astype(np.float32)
    opts['mny'] = opts['moneyness'].astype(np.float32)
    opts['iv'] = opts['Implied Volatility'].astype(np.float32) / 100
    
    # filtering
    opts = opts.loc[(~opts['iv'].isna()) & 
                    (opts['T']>=20) & (opts['T']<=365) & 
                    (opts['mny']>0.7) & (opts['mny']<1.3)]


    # split up per day
    for quote_date in opts['quote'].unique():
    
        # splitting up into calls/puts
        calls = opts.loc[(opts['quote'] == quote_date) & (opts['option_type']=='c')][['T', 'mny', 'iv']]
        puts = opts.loc[(opts['quote'] == quote_date) & (opts['option_type']=='p')][['T', 'mny', 'iv']]
        
        # ensure enough data        
        if len(calls) < 10 or len(puts) < 10:
            continue
        
        calls_puts = {'calls':calls, 'puts':puts}
        daily_chains[f'{quote_date.date()}'] = calls_puts

        

In [ ]:
# sort daily_chains
daily_chains = dict(sorted(daily_chains.items()))

### GP DEFINITION

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

### TRAINING

In [ ]:
def custom_split(df, n_bins, test_size=0.2):
    
    # bin moneyness
    grouped = df.groupby(['T'])
    
    train_dfs = []
    test_dfs = []
    
    for _, group in grouped:
        # if not enough data
        if len(group)//n_bins <= 1/test_size:
            train, test = train_test_split(group, test_size=test_size)
            
        # split into bins
        else:
            group['bins'] = pd.qcut(group['mny'], n_bins, duplicates='drop', labels=False)
            train, test = train_test_split(group, test_size=test_size, stratify=group['bins'])

        train_dfs.append(train)
        test_dfs.append(test)
    
    train_df = pd.concat(train_dfs)
    test_df = pd.concat(test_dfs)
    
    return train_df, test_df


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="is_sparse is deprecated and will be removed in a future version")

gp_models = {}

for day, options in list(daily_chains.items()):
    
    info = {}
    
    # calls and puts
    c = options['calls']
    p = options['puts']
        
    # feature transformations
    c['mm_T'] = (c['T'] - 20)/(365-20)
    c['mm_mny'] = (c['mny'] - 0.7)/(1.3-0.7)
    c['ln_iv'] = np.log(c['iv'])

    p['mm_T'] = (p['T'] - 20)/(365-20)
    p['mm_mny'] = (p['mny'] - 0.7)/(1.3-0.7)
    p['ln_iv'] = np.log(p['iv'])
    
    # test/train split
    c_train, c_test = custom_split(c, 3, test_size=0.2)
    p_train, p_test = custom_split(p, 3, test_size=0.2)
    info['call_train'] = c_train
    info['call_test'] = c_test
    info['put_train'] = p_train
    info['put_test'] = p_test
    
    # into tensors
    cx_train = torch.tensor(c_train[['mm_T', 'mm_mny']].values)
    cy_train = torch.tensor(c_train[['ln_iv']].values).reshape(len(c_train))
    cx_test = torch.tensor(c_test[['mm_T', 'mm_mny']].values)
    cy_test = torch.tensor(c_test[['ln_iv']].values).reshape(len(c_test))
    
    px_train = torch.tensor(p_train[['mm_T', 'mm_mny']].values)
    py_train = torch.tensor(p_train[['ln_iv']].values).reshape(len(p_train))
    px_test = torch.tensor(p_test[['mm_T', 'mm_mny']].values)
    py_test = torch.tensor(p_test[['ln_iv']].values).reshape(len(p_test))
    
    #break
    # initializing likelihood and model
    c_likelihood = gpytorch.likelihoods.GaussianLikelihood()
    c_model = ExactGPModel(cx_train, cy_train, c_likelihood)  
    
    p_likelihood = gpytorch.likelihoods.GaussianLikelihood()
    p_model = ExactGPModel(px_train, py_train, p_likelihood)  
    
    # finding optimal model parameters
    c_model.train()
    c_likelihood.train()

    p_model.train()
    p_likelihood.train()
    
    # Use the adam optimizer
    c_optimizer = torch.optim.Adam(c_model.parameters(), lr=0.05) 
    p_optimizer = torch.optim.Adam(p_model.parameters(), lr=0.05) 

    # "Loss" for GPs - the marginal log likelihood
    c_mll = gpytorch.mlls.ExactMarginalLogLikelihood(c_likelihood, c_model)
    p_mll = gpytorch.mlls.ExactMarginalLogLikelihood(p_likelihood, p_model)
    c_losses = []
    p_losses = []
    
    training_iter = 250
    print('STARTING CALL GP TRAINING FOR ', day)
    for i in range(training_iter):
        # Zero gradients from previous iteration
        c_optimizer.zero_grad()
        # Output from model
        c_output = c_model(cx_train)
        # Calc loss and backprop gradients
        c_loss = -c_mll(c_output, cy_train)
        c_loss.backward()
        
        c_losses.append(c_loss.item())
        if i % 5 == -1:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, c_loss.item(),
                c_model.covar_module.base_kernel.lengthscale.item(),
                c_model.likelihood.noise.item()
            ))
        c_optimizer.step()
        
    print('STARTING PUT GP TRAINING FOR ', day)
    for i in range(training_iter):
        # Zero gradients from previous iteration
        p_optimizer.zero_grad()
        # Output from model
        p_output = p_model(px_train)
        # Calc loss and backprop gradients
        p_loss = -p_mll(p_output, py_train)
        p_loss.backward()
        
        p_losses.append(p_loss.item())
        if i % 5 == -1:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, p_loss.item(),
                p_model.covar_module.base_kernel.lengthscale.item(),
                p_model.likelihood.noise.item()
            ))
        p_optimizer.step()
    
    info['call_losses'] = c_losses
    info['put_losses'] = p_losses

    # Get into evaluation (predictive posterior) mode
    c_model.eval()
    c_likelihood.eval()
    
    p_model.eval()
    p_likelihood.eval()

    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        c_preds = c_likelihood(c_model(cx_test))
        p_preds = p_likelihood(p_model(px_test))

    info['call_preds'] = c_preds
    info['put_preds'] = p_preds        

    # rmse
    c_rmse = torch.sqrt(torch.mean(torch.pow(math.e ** c_preds.mean - math.e ** cy_test, 2)))
    p_rmse = torch.sqrt(torch.mean(torch.pow(math.e ** p_preds.mean - math.e ** py_test, 2)))
    info['call_RMSE'] = c_rmse
    info['put_RMSE'] = p_rmse
    
    # saving the model
    torch.save(c_model, 'e_models/call_GP_'+day+'.pt')
    torch.save(p_model, 'e_models/put_GP_'+day+'.pt')
    
    # saving likelihood
    info['call_likelihood'] = c_likelihood
    info['put_likelihood'] = p_likelihood
    
    gp_models[day] = info  
        

In [ ]:
# debug
for day in daily_chains.keys():
    c = daily_chains[day]['calls']
    p = daily_chains[day]['puts']
    try:
        custom_split(c, 3, 0.2)
        custom_split(p, 3, 0.2)
    except:
        print(day)

### PLOTTING LOSSES

In [ ]:
# finding time ot expirations for a specific day
day_str = '2023-10-02'
print('valid times to exp:', sorted(daily_chains[day_str]['calls']['T'].unique()))

In [ ]:
# plotting losses
c_plt_loss = gp_models[day_str]['call_losses']
p_plt_loss = gp_models[day_str]['put_losses']

plt.plot(range(len(c_plt_loss)), c_plt_loss, label='call loss')
plt.plot(range(len(p_plt_loss)), p_plt_loss, label='put loss')
plt.legend()
plt.show()

### PLOTTING VOL SMILE

In [ ]:
t_to_exp = 354

# getting training/test data
c_tr = gp_models[day_str]['call_train']
c_tr = c_tr.loc[c_tr['T']==t_to_exp]
c_tst = gp_models[day_str]['call_test']
c_tst = c_tst.loc[c_tst['T']==t_to_exp]

p_tr = gp_models[day_str]['put_train']
p_tr = p_tr.loc[p_tr['T']==t_to_exp]
p_tst = gp_models[day_str]['put_test']
p_tst = p_tst.loc[p_tst['T']==t_to_exp]

# getting model/likelihood
c_plt_model = torch.load('e_models/call_GP_'+day_str+'.pt')
c_plt_model.eval()
p_plt_model = torch.load('e_models/put_GP_'+day_str+'.pt')
p_plt_model.eval()
c_ll = gp_models[day_str]['call_likelihood']
c_ll.eval()
p_ll = gp_models[day_str]['put_likelihood']
p_ll.eval()

# sample data
rng_mny = np.array(np.linspace(0.7, 1.3, 100)).astype(np.float32)
samp_mny = (rng_mny - 0.7) / (1.3 - 0.7)
t_arr = np.array([t_to_exp]*100).astype(np.float32)
samp_t = (t_arr - 20) / (365 - 20)

sample = torch.tensor(np.array([samp_t, samp_mny])).T #.reshape((1000, 2))

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    c_plt = c_ll(c_plt_model(sample))
    p_plt = p_ll(p_plt_model(sample))
    
# make dumas vol surface as well
c_dumas_model = dumas_surface(c_tr.copy(deep=True))
p_dumas_model = dumas_surface(p_tr.copy(deep=True))

c_dumas_preds = c_dumas_model.predict(np.array([rng_mny, rng_mny**2, t_arr, t_arr**2, rng_mny*t_arr]).T)
p_dumas_preds = p_dumas_model.predict(np.array([rng_mny, rng_mny**2, t_arr, t_arr**2, rng_mny*t_arr]).T)

c_dumas = pd.DataFrame({'mny':rng_mny, 'T':t_arr, 'iv':c_dumas_preds})
p_dumas = pd.DataFrame({'mny':rng_mny, 'T':t_arr, 'iv':p_dumas_preds})

In [ ]:
# calls
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(8, 6))

    # Get upper and lower confidence bounds
    lower, upper = c_plt.confidence_region()
    
    # Plot training data as black stars
    ax.plot(c_tr['mny'], c_tr['iv'], 'k*', label='training points')
    
    # Plot testing data as blue dots
    ax.plot(c_tst['mny'], c_tst['iv'], 'b.', label='testing points')
    
    # undo exponentiation
    lower = math.e ** lower.numpy() #np.log(lower.numpy())
    upper = math.e ** upper.numpy() #np.log(upper.numpy() ) 
    preds = math.e ** c_plt.mean.numpy() #np.log(c_plt.mean.numpy())
    
    # undo x-axis scaling by using rng_mny
    ax.plot(rng_mny, preds, 'r', label='mean function')
    
    # plot dumas
    #ax.plot(rng_mny, c_dumas_preds, 'g', label='Dumas IV')
    
    # Shade between the lower and upper confidence bounds
    ax.fill_between(rng_mny, lower, upper, alpha=0.5, label='95% confidence')
    
    ax.set_xlim([0.7, 1.3])
    ax.set_xlabel("Moneyness")
    ax.set_ylabel("Implied Volatility")
    ax.set_title(f"Call IV Surface for a Time to Expiration of {t_to_exp} days")
    ax.legend()


In [ ]:
# puts
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(8, 6))

    # Get upper and lower confidence bounds
    lower, upper = p_plt.confidence_region()
    
    # Plot training data as black stars
    ax.plot(p_tr['mny'], p_tr['iv'], 'k*', label='training points')
    
    # plot test data as blue stars
    ax.plot(p_tst['mny'], p_tst['iv'], 'b.', label='testing points')
    
    # undo exponentiation
    lower = math.e ** lower.numpy() #np.log(lower.numpy())
    upper = math.e ** upper.numpy() #np.log(upper.numpy() ) 
    preds = math.e ** p_plt.mean.numpy() #np.log(p_plt.mean.numpy())
    
    # undo x-axis scaling by using original range
    ax.plot(rng_mny, preds, 'r', label='mean function')
    #ax.plot(rng_mny, p_dumas_preds, 'g', label='Dumas IV')

    
    # Shade between the lower and upper confidence bounds
    ax.fill_between(rng_mny, lower, upper, alpha=0.5, label='95% confidence')
    
    ax.set_xlim([0.7, 1.3])
    #ax.set_ylim([)
    ax.set_xlabel("Moneyness")
    ax.set_ylabel("Implied Volatility")
    ax.set_title(f"Put IV Surface for a Time to Expiration of {t_to_exp} days")

    ax.legend()


In [ ]:
print(f'call rmse for {day_str}: {gp_models[day_str]["call_RMSE"]}')
print(f'put rmse for {day_str}: {gp_models[day_str]["put_RMSE"]}')

In [ ]:
c_rmse_arr = []
p_rmse_arr = []
for day in list(daily_chains.keys()):
    c_rmse_arr.append(gp_models[day]["call_RMSE"])
    p_rmse_arr.append(gp_models[day]["put_RMSE"])
    
print(f'mean call rmse: {np.array(c_rmse_arr).mean()}')
print(f'max call rmse: {np.array(c_rmse_arr).max()}')
print(f'min call rmse: {np.array(c_rmse_arr).min()}')
print()
print(f'mean put rmse: {np.array(p_rmse_arr).mean()}')
print(f'max put rmse: {np.array(p_rmse_arr).max()}')
print(f'min put rmse: {np.array(p_rmse_arr).min()}')


In [ ]:
c_surface = [] # coord (T, mny, iv)
p_surface = []

valid_t = daily_chains[day_str]['calls']['T'].unique()

# getting model/likelihood
c_plt_model = torch.load('e_models/call_GP_'+day_str+'.pt')
c_plt_model.eval()
p_plt_model = torch.load('e_models/put_GP_'+day_str+'.pt')
p_plt_model.eval()
c_ll = gp_models[day_str]['call_likelihood']
c_ll.eval()
p_ll = gp_models[day_str]['put_likelihood']
p_ll.eval()

rng_mny = np.array(np.linspace(0.7, 1.3, 100)).astype(np.float32)
samp_mny = (rng_mny - 0.7) / (1.3 - 0.7)

for t_to_exp in valid_t:

    # getting training/test data
    c_tr = gp_models[day_str]['call_train']
    c_tr = c_tr.loc[c_tr['T']==t_to_exp]
    c_tst = gp_models[day_str]['call_test']
    c_tst = c_tst.loc[c_tst['T']==t_to_exp]

    p_tr = gp_models[day_str]['put_train']
    p_tr = p_tr.loc[p_tr['T']==t_to_exp]
    p_tst = gp_models[day_str]['put_test']
    p_tst = p_tst.loc[p_tst['T']==t_to_exp]

    # sample data
    t_arr = np.array([t_to_exp]*100).astype(np.float32)
    samp_t = (t_arr - 20) / (365 - 20)

    sample = torch.tensor([samp_t, samp_mny]).T #.reshape((1000, 2))
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        c_plt = c_ll(c_plt_model(sample))
        p_plt = p_ll(p_plt_model(sample))
    
    # iv
    c_func = math.e ** c_plt.mean.numpy()
    p_func = math.e ** p_plt.mean.numpy()
    
    for i in range(100):
        c_surface.append([rng_mny[i], t_arr[i], c_func[i]])
        p_surface.append([rng_mny[i], t_arr[i], p_func[i]])
        
c_surface = np.array(c_surface)
p_surface = np.array(p_surface)

    

In [ ]:
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

Xs = c_surface[:,0]
Ys = c_surface[:,1]
Zs = c_surface[:,2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_trisurf(Xs, Ys, Zs, cmap=cm.jet, linewidth=0)
fig.colorbar(surf, label='Implied Volatility')

ax.set_xlabel('moneyness')
ax.set_ylabel('time to expiration')

ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(6))
ax.zaxis.set_major_locator(MaxNLocator(5))
ax.set_title('Call Volatility Surface')

fig.tight_layout()

plt.show() # or:

In [ ]:
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

Xs = p_surface[:,0]
Ys = p_surface[:,1]
Zs = p_surface[:,2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_trisurf(Xs, Ys, Zs, cmap=cm.jet, linewidth=0)
fig.colorbar(surf, label='Implied Volatility')

ax.set_xlabel('moneyness')
ax.set_ylabel('time to expiration')

ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(6))
ax.zaxis.set_major_locator(MaxNLocator(5))
ax.set_title('Put Volatility Surface')


fig.tight_layout()

plt.show() # or:

### DUMAS

In [ ]:
c_dumas_rmse = []
p_dumas_rmse = []

for day_str in list(daily_chains.keys()):
    
    c_tr = gp_models[day_str]['call_train']
    p_tr = gp_models[day_str]['put_train']
    c_tst = gp_models[day_str]['call_test']
    p_tst = gp_models[day_str]['put_test']
        
    # make dumas vol surface as well
    c_dumas_model = dumas_surface(c_tr.copy(deep=True))
    p_dumas_model = dumas_surface(p_tr.copy(deep=True))
    
    cmny = c_tst['mny'].values
    c_t = c_tst['T'].values
    
    pmny = p_tst['mny'].values
    p_t = p_tst['T'].values
    
    c_dumas_preds = c_dumas_model.predict(np.array([cmny, cmny**2, c_t, c_t**2, cmny*c_t]).T)
    p_dumas_preds = p_dumas_model.predict(np.array([pmny, pmny**2, p_t, p_t**2, pmny*p_t]).T)
    
    c_rmse = np.sqrt(np.mean((c_dumas_preds - c_tst['iv'].values)**2))
    p_rmse = np.sqrt(np.mean((p_dumas_preds - p_tst['iv'].values)**2))

    c_dumas_rmse.append(c_rmse)
    p_dumas_rmse.append(p_rmse)


In [ ]:
c_rmse_arr = []
p_rmse_arr = []
for day in list(daily_chains.keys()):
    c_rmse_arr.append(gp_models[day]["call_RMSE"])
    p_rmse_arr.append(gp_models[day]["put_RMSE"])

print('dumas:', np.mean(c_dumas_rmse), np.mean(p_dumas_rmse))
print('gp:', np.mean(c_rmse_arr), np.mean(p_rmse_arr))

### PRICING

In [ ]:
import scipy.stats as ss

def bs(S0, K, T, r, sigma, payoff):
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))

    if payoff=="call":
        return S0 * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
    elif payoff=="call":
        return K * np.exp(-r * T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)
    else:
        return -1


def get_rfr(mp, S0, K, T, sigma, payoff):
    
    # binary search between 0 and 1
    min_rfr = 0
    max_rfr = 1
    rfr = 0.5
    while max_rfr - min_rfr > 0.001:
        bsp = bs(S0, K, T, rfr, sigma, payoff)
        if bsp < mp:
            max_rfr = rfr
            rfr = (rfr + min_rfr) / 2
        else:
            min_rfr = rfr
            rfr = (rfr + max_rfr) / 2
    
    return rfr

def model_price(mp, S0, K, T, sigma, pred, payoff):
    
    rfr = get_rfr(mp, S0, K, T, sigma, payoff)
    return bs(S0, K, T, rfr, pred, payoff)
    

In [ ]:
### Derive risk-free rate and then price with predicted IV

for day_str in list(daily_chains.keys())[:1]:

    c_tst = gp_models[day_str]['call_test']
    c = daily_chains[day_str]['calls']
    p_tst = gp_models[day_str]['put_test']
    p = daily_chains[day_str]['puts']
    
    c_tst['midprice'] = c.loc[c_tst.index, 'midprice']
    c_tst['underlying'] = c.loc[c_tst.index, 'underlying']
    p_tst['midprice'] = p.loc[p_tst.index, 'midprice']
    p_tst['underlying'] = p.loc[p_tst.index, 'underlying']
    c_tst['preds'] = gp_models[day_str]['call_preds'].mean.numpy()
    p_tst['preds'] = gp_models[day_str]['put_preds'].mean.numpy()
    
    
    cprices = c_tst.apply(lambda x: model_price(x['midprice'], x['underlying'], x['strike'], x['T'], x['iv'], x['pred'], 'call'), axis=1)
    pprices = p_tst.apply(lambda x: model_price(x['midprice'], x['underlying'], x['strike'], x['T'], x['iv'], x['pred'], 'put'), axis=1)
    
    
    
    


In [ ]:
# save daily_chain to pickles/chain_info.pickle 
with open('pickles/chain_info.pickle', 'wb') as handle:
    pickle.dump(daily_chains, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pickles/model_info.pickle', 'wb') as handle:
    pickle.dump(gp_models, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:

daily_chains['2022-02-04']['calls']